In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json
from bs4 import BeautifulSoup
### ML Librarires--------------------
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
###-------------------------------------------
#from keras.utils.vis_utils import plot_model
np.random.seed(42)
import string, spacy,unicodedata, random
class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
import warnings
warnings.filterwarnings('ignore')


In [ ]:
!pip install keras_preprocessing
from keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, GlobalAveragePooling2D, Embedding, LSTM, multiply
from keras.models import Model
from keras import preprocessing, Input
from tensorflow import keras
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras.models import load_model
import itertools
from PIL import Image, ImageFile
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten, Reshape, Permute
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D,MaxPooling1D,Conv1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Add, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Nadam

In [ ]:
# import pandas as pd

# # Change the path to your actual data location
# data_path = "/content/drive/MyDrive/Dataset/multi-sent.xlsx"

# # Read the data into a Pandas DataFrame
# data = pd.read_excel(data_path)

# # Print the column names
# print(data.columns)

In [ ]:
# Locate images from /content/drive/MyDrive/CM_MEMES-master/LREC_COLING_meme
image_folder = "/content/drive/MyDrive/Dataset/Memes"
image_names = set(os.listdir(image_folder))

# Import sample_data.csv from /content/drive/MyDrive/CM_MEMES-master/sample_data.csv
csv_file_path = "/content/drive/MyDrive/Dataset/multi-sent.xlsx"
data = pd.read_excel(csv_file_path)

# Cross-check all the image names with the Name column and remove rows that don't exist
data = data[data["image_name"].isin(image_names)]

# Display the updated DataFrame
print(data)

In [ ]:
data.head()

In [ ]:
data = data.rename(columns={"Label_Sentiment": "label"})

In [ ]:
# Randomize data with seed
data = data.sample(frac=1, random_state=42)

# Split data into train and test sets (80:20)
train_size = int(0.8 * len(data))
train_data, test_data = data[:train_size], data[train_size:]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
train_data.to_csv('/content/drive/MyDrive/Dataset/train_data.csv', index=False)
test_data.to_csv('/content/drive/MyDrive/Dataset/test_data.csv', index=False)

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Dataset/train_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Dataset/test_data.csv')

print("Number of Training Data: ", len(train_data))
print("Number of Test Data: ", len(test_data))

In [ ]:
train_data['label'] = train_data['label'].replace({'positive':0,'negative':1,'neutral':2})
test_data['label'] = test_data['label'].replace({'positive':0,'negative':1,'neutral':2})

In [ ]:
memes_path = "/content/drive/MyDrive/Dataset/Memes/"

In [ ]:
## collect image names from the folders
def create_img_array(img_dirct):
    all_imgs = []
    for root, j, files in os.walk(img_dirct):
        for file in files:
            file = root + '' + file
            all_imgs.append(file)
    return all_imgs

def create_img_path(DF, Col_name, img_dir):
    img_path = [img_dir + '' + name for name in DF[Col_name]]
    return img_path

In [ ]:
# Function that returns image reading from the path
def get_input(path):
    # Loading image from given path
    # and resizing it to 150*150*3 format
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    img = tf.keras.utils.load_img(path, target_size=(150,150))
    return(img)

# Takes in image and preprocess it
def process_input(img):
    # Converting image to array
    img_data = tf.keras.utils.img_to_array(img)
    # Adding one more dimension to array
    img_data = np.expand_dims(img_data, axis=0)
    #
    img_data = preprocess_input(img_data)
    return(img_data)


In [ ]:
img_dir = memes_path
# Creating train, test and validation image path
train_img_path = create_img_path(train_data,'image_name', img_dir)
test_img_path = create_img_path(test_data,'image_name', img_dir)

In [ ]:
# Create an array of training images
train_images = []
for i,m in enumerate(train_img_path):
  input_img = get_input(m)
  input_img = process_input(input_img)
  train_images.append(input_img[0])
  print(i)

# convert into numpy array
train_image = np.array(train_images)
print(train_image.shape)

In [ ]:
main_path = '/content/drive/MyDrive/Dataset'

In [ ]:
import pickle as pkl
with open(main_path+'train_image_new.pkl','wb') as f:
    pkl.dump(train_image, f)

In [ ]:
# Create an array of test images
test_images = []
for i,m in enumerate(test_img_path):
  input_img = get_input(m)
  input_img = process_input(input_img)
  test_images.append(input_img[0])
  print(i)

# convert into numpy array
test_image = np.array(test_images)
print(test_image.shape)

In [ ]:
import pickle as pkl
with open(main_path+'test_image_new.pkl','wb') as f:
    pkl.dump(test_image, f)

In [ ]:
import pickle as pkl

with open(main_path+'train_image_new.pkl','rb') as f:
  train_image = pkl.load(f)
  print("Training Images:-- ",train_image.shape)

with open(main_path+'test_image_new.pkl','rb') as f:
  test_image = pkl.load(f)
  print("Test Images:-- ",test_image.shape)

In [ ]:
'''
Text Cleaning
'''
def text_cleaning(row):
   #to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  post = post.replace('।', ' ')
  text = ''.join([c for c in post if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower()
  # # Stopword
  # result = text.split()
  # text = [word.strip() for word in result if word not in stp ]
  # text =" ".join(text)

  return text

In [ ]:
#Removing punctuations
train_data['cleaned'] = train_data['Captions'].apply(text_cleaning)
test_data['cleaned'] = test_data['Captions'].apply(text_cleaning)


## Data samples after cleaning
print("Data samples after cleaning:\n")
for i in range(100):
  print("Original Data:===\n",train_data.Captions[i],"\nCleaned Data:===\n",train_data.cleaned[i],)

In [ ]:
train_data.head()

In [ ]:
def data_summary(dataset):
    documents = []
    words = []
    u_words = []
    class_label= [k for k,v in dataset.label.value_counts().to_dict().items()]
  # find word list
    for label in class_label:
        word_list = [word.strip().lower() for t in list(dataset[dataset.label==label].cleaned) for word in t.strip().split()]
        counts = dict()
        for word in word_list:
                counts[word] = counts.get(word, 0)+1
    # sort the dictionary of word list
        ordered = sorted(counts.items(), key= lambda item: item[1],reverse = False)
    # Documents per class
        documents.append(len(list(dataset[dataset.label==label].cleaned)))
    # Total Word per class
        words.append(len(word_list))
    # Unique words per class
        u_words.append(len(np.unique(word_list)))

        print("\nClass Name : ",label)
        print("Number of Comments:{}".format(len(list(dataset[dataset.label==label].cleaned))))
        print("Number of Words:{}".format(len(word_list)))
        print("Number of Unique Words:{}".format(len(np.unique(word_list))))
        print("Most Frequent Words:\n")
        for k,v in ordered[:50]:
              print("{}\t{}".format(k,v))

    return documents,words,u_words,class_label

In [ ]:
documents,words,u_words,class_names = data_summary(train_data)

**Eval**

In [ ]:
def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,target_names=['0','1']))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

In [ ]:
models_path = "/content/drive/MyDrive/Dataset/model/"

In [ ]:
''' Callbacks'''
keras.backend.clear_session()
def callbacks_check(model_name):
  accuracy_threshold = 0.98

  class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>accuracy_threshold):
          print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
          self.model.stop_training = True

  acc_callback = myCallback()
  # Saved the Best Model
  filepath = models_path+f"{model_name}.h5"
  checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True,
                                             save_weights_only=False, mode='max')
  # callback list
  callback_list = [acc_callback, checkpoint]

  return callback_list

In [ ]:
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_data['label']),
                                        y = train_data['label']
                                    )
class_weights = dict(zip(np.unique(train_data['label']), class_weights))
class_weights

In [ ]:
def visual_models(pretrained_model):
  '''
      Input: Pretrained Models weight
  '''
  base_model = pretrained_model
  base_model.trainable = False
  y = base_model.output
  pool = GlobalAveragePooling2D()(y)
  #flatten = Flatten()(pool)
  output = Dense(3, activation='softmax')(pool)
  # train model
  img_model = Model(inputs=base_model.input, outputs=output)

  return img_model

In [ ]:
xception = Xception(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
inception = InceptionV3(weights='imagenet', include_top=False,input_shape=(150, 150, 3)) #added inception
vgg19 = VGG19(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
vgg16 = VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
densenet = DenseNet121(weights='imagenet', include_top=False,input_shape=(150, 150, 3))

In [ ]:
xcept_model = visual_models(xception)
xcept_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=Adam(),
                    metrics = ["accuracy"])

In [ ]:
keras.backend.clear_session()

In [ ]:
xcept_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              #class_weight = weight,
              callbacks = callbacks_check('xception')
              )

In [ ]:
 # Prediction
model = load_model(models_path+"xception.h5")
pred = model.predict(test_image)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
inception_model = visual_models(inception)
inception_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=Adam(),
                    metrics = ["accuracy"])

In [ ]:
keras.backend.clear_session()
inception_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              callbacks = callbacks_check('inception')
              )

In [ ]:
#prediction
model = load_model(models_path+"inception.h5")
pred = model.predict(test_image)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
vgg19_model = visual_models(vgg19)
vgg19_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=Adam(),
                    metrics = ["accuracy"])

In [ ]:
keras.backend.clear_session()
vgg19_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              callbacks = callbacks_check('vgg19')
              )

In [ ]:
 # Prediction
model = load_model(models_path+"vgg19.h5")
pred = model.predict(test_image)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
keras.backend.clear_session()
vgg16_model = visual_models(vgg16)
vgg16_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=RMSprop(),
                    metrics = ["accuracy"])

In [ ]:
vgg16_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              callbacks = callbacks_check('vgg16')
              )

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
keras.backend.clear_session()
resnet_model = visual_models(resnet)
resnet_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=RMSprop(),
                    metrics = ["accuracy"])

In [ ]:
resnet_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              callbacks = callbacks_check('resnet-1')
              )

In [ ]:
 # Prediction
model = load_model(models_path+"resnet.h5")
pred = model.predict(test_image)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
keras.backend.clear_session()
densenet_model = visual_models(densenet)
densenet_model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=Adam(),
                    metrics = ["accuracy"])

In [ ]:
densenet_model.fit(x=train_image,
              y=train_data['label'],
              epochs=20,
              batch_size =32,
              validation_split = 0.1,
              verbose = 1,
              callbacks = callbacks_check('densenet')
              )

In [ ]:
 # Prediction
model = load_model(models_path+"densenet.h5")
pred = model.predict(test_image)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)

In [ ]:
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]

In [ ]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
visual_models = ['xception','inception','vgg19','vgg16','resnet','densenet']
visual_model_names = ['Xception','Inception','vgg19','Vgg16','Resnet','Densenet']

def visual_models_accuracy(saved_model):
  my_dict = {}
  # Prediction
  model = load_model(models_path+f"{saved_model}.h5")
  pred = model.predict(test_image)
  y_pred = np.argmax(pred,axis=1)

  y_true = test_data['label']

  my_dict['Accuracy'] = accuracy_score(y_true, y_pred)*100
  my_dict['Precision'] = precision_score(y_true, y_pred,average = 'weighted')*100
  my_dict['Recall'] = recall_score(y_true, y_pred,average = 'weighted')*100
  my_dict['F1 Score'] = f1_score(y_true, y_pred,average = 'weighted')*100
  return my_dict

In [ ]:
results_path = '/content/drive/MyDrive/Dataset/results/'

accuracy = {f'{visual_model_names[i]}':visual_models_accuracy(model) for i,model in enumerate(visual_models)}
# Save the performance parameter into json file
with open(results_path+'visual_models_performance.json', 'w') as f:
    json.dump(accuracy, f)

In [ ]:
# Load the json file
metrics = json.load(open(results_path+'visual_models_performance.json'))
acc_list = []
pr_list = []
re_list = []
f1_list = []
for i in metrics.keys():
  acc_list.append(round(metrics[i]['Accuracy'],2))
  pr_list.append(round(metrics[i]['Precision'],2))
  re_list.append(round(metrics[i]['Recall'],2))
  f1_list.append(round(metrics[i]['F1 Score'],2))

print (color.BOLD+f"=======  Visual Models Performance on Test Data  =============\n"+color.END)
# Create a dataframe
performance_matrix = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =['Xception', 'Inception','Vgg19','Vgg16','Resnet','Densenet'])
performance_matrix

**Text**

In [ ]:
# Text Feature Extraction using TF-IDF
cv = TfidfVectorizer(use_idf=True,tokenizer=lambda x: x.split())
train_X = cv.fit_transform(train_data.Captions)

# Test data
test_X = cv.transform(test_data.Captions)

print("\nShape of TF-IDF Corpus =====>",train_X.shape)
print("\nShape of TF-IDF Corpus =====>",test_X.shape)

In [ ]:
lr_model = LogisticRegression(random_state = 123)
dt_model = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
rf_model = RandomForestClassifier(n_estimators=10, criterion ='entropy', random_state = 0)
mnb_model = MultinomialNB(alpha=0.05)
svm_model = SVC(kernel = 'linear',probability=True,random_state = 0)
model_names = ['Logistic Regression','Naive Bayes','SVM']
ml_models = [lr_model,mnb_model,svm_model]

def model_accuracy(model,X_train,X_test,y_train,y_test):
  my_dict = {}
  model.fit(X_train,y_train)
  # Prediction
  pred_y = model.predict(X_test)
  my_dict['Accuracy'] = round(accuracy_score(y_test, pred_y),3)*100
  my_dict['Precision'] = round(precision_score(y_test, pred_y,average='weighted'),3)*100
  my_dict['Recall'] = round(recall_score(y_test, pred_y,average='weighted'),3)*100
  my_dict['F1 Score'] = round(f1_score(y_test, pred_y,average='weighted'),3)*100
  return my_dict

# call model accuracy function and save the metrices into a dictionary
accuracy = {f'{model_names[i]}':model_accuracy(model,train_X,test_X,train_data['label'],test_data['label']) for i,model in enumerate(ml_models)}
# Save the performance parameter into json file
with open(results_path +'ml_model_performance.json', 'w') as f:
    json.dump(accuracy, f)

In [ ]:
# Load the json file
accuracy = json.load(open(results_path +'ml_model_performance.json'))
acc_list = []
pr_list = []
re_list = []
f1_list = []
for i in accuracy.keys():
  acc_list.append(accuracy[i]['Accuracy'])
  pr_list.append(accuracy[i]['Precision'])
  re_list.append(accuracy[i]['Recall'])
  f1_list.append(accuracy[i]['F1 Score'])

# Create a dataframe
performance_matrix = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =['LR','MNB','SVM'])
performance_matrix

In [ ]:
def text_tokenizer(train_data,test_data,vocabulary,max_len,sample_text_num):

  tokenizer = Tokenizer(num_words = vocabulary ,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-',
                        split=' ', char_level=False, oov_token='<oov>', document_count=0)
  tokenizer.fit_on_texts(train_data['Captions'])
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  # Training Sequences
  train_sequences = tokenizer.texts_to_sequences(train_data['Captions'])
  train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)

  # valid Sequences
  # valid_sequences = tokenizer.texts_to_sequences(valid_data['Captions'])
  # valid_pad_sequences =  keras.preprocessing.sequence.pad_sequences(valid_sequences, value=0.0, padding='post', maxlen= max_len)


  # Test Sequences
  test_sequences = tokenizer.texts_to_sequences(test_data['Captions'])
  test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)


  print(color.BOLD+"\n\t\t\t====== Encoded Sequences ======"+color.END,"\n")
  print(train_data.Captions[sample_text_num],"\n",train_sequences[sample_text_num])
  print(color.BOLD+"\n\t\t\t====== Paded Sequences ======\n"+color.END,"\n",train_pad_sequences[sample_text_num])

  return train_pad_sequences,  test_pad_sequences, vocab_size, word_index

  # return train_pad_sequences, valid_pad_sequences, test_pad_sequences, vocab_size, word_index


vocabulary = 15000
max_len = 60
sample_text_num = 10

## Call Tokenizer
# train_pad_sequences, valid_pad_sequences, test_pad_sequences, vocab_size, word_index =  text_tokenizer(train_data,test_data,
#                                                                       vocabulary,max_len,sample_text_num)

train_pad_sequences,  test_pad_sequences, vocab_size, word_index =  text_tokenizer(train_data,test_data,
                                                                      vocabulary,max_len,sample_text_num)

print("Number of Train Sequences :" ,train_pad_sequences.shape)
# print("Number of Test Sequences :" ,valid_pad_sequences.shape)
print("Number of Test Sequences :" ,test_pad_sequences.shape)
print("Vocabulary Size: ",vocab_size)

In [ ]:
###### BiLSTM Model #######
bi_text_inputs = Input(shape=(max_len,))
bi_embedding_layer = Embedding(vocab_size,64)(bi_text_inputs)
LSTM_Layer_1 = Bidirectional(LSTM(32))(bi_embedding_layer)
bi_dense_layer_1 = Dense(3, activation='softmax')(LSTM_Layer_1)
bilstm_model = Model(inputs=bi_text_inputs, outputs=bi_dense_layer_1)

In [ ]:
keras.backend.clear_session()
bilstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


bilstm_model.fit(train_pad_sequences,
    train_data['label'],
    epochs= 20,
    batch_size =32,
    validation_split = 0.1,
    verbose =1,
    callbacks = callbacks_check('lstm'))

In [ ]:
 # Prediction
model = load_model(models_path + "lstm.h5")
pred = model.predict(test_pad_sequences)
y_pred = np.argmax(pred,axis=1)

In [ ]:
unique_labels = y_true.unique()
num_classes = len(unique_labels)
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
### CNN
cnn_text_inputs = Input(shape=(max_len,))
cnn_embedding_layer = Embedding(vocab_size,64)(cnn_text_inputs)
cnn_conv1 = Conv1D(64,2,activation='relu')(cnn_embedding_layer)
cnn_conv2 = Conv1D(32,2,activation='relu')(cnn_conv1)
cnn_pool1 = MaxPooling1D(2)(cnn_conv2)

cnn_flat = Flatten()(cnn_pool1)
cnn_dense_layer_1 = Dense(3, activation='softmax')(cnn_flat)
cnn_model = Model(inputs=cnn_text_inputs, outputs=cnn_dense_layer_1)

In [ ]:
keras.backend.clear_session()
cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


cnn_model.fit(train_pad_sequences,
    train_data['label'],
    epochs=20,
    batch_size =32,
    validation_split = 0.1,
    verbose =1,
    callbacks = callbacks_check('cnn'))

In [ ]:
 # Prediction
model = load_model(models_path+"cnn.h5")
pred = model.predict(test_pad_sequences)
y_pred = np.argmax(pred,axis=1)
unique_labels = y_true.unique()
num_classes = len(unique_labels)
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
##### LSTM-CNN Model ####
lc_text_inputs = Input(shape=(max_len,))
lc_embedding_layer = Embedding(vocab_size,64)(lc_text_inputs)
LSTM_Layer = Bidirectional(LSTM(32,return_sequences=True))(lc_embedding_layer)
lc_conv1 = Conv1D(32,2,activation='relu')(LSTM_Layer)
lc_pool1 = MaxPooling1D(2)(lc_conv1)
lc_flat = Flatten()(lc_pool1)
lc_dense_layer_1 = Dense(3, activation='softmax')(lc_flat)
cnn_lstm_model = Model(inputs=lc_text_inputs, outputs=lc_dense_layer_1)

In [ ]:
keras.backend.clear_session()
cnn_lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


cnn_lstm_model.fit(train_pad_sequences,
    train_data['label'],
    epochs=20,
    batch_size =32,
    validation_split = 0.1,
    verbose =1,
    callbacks = callbacks_check('lstm-cnn'))

In [ ]:
 # Prediction
model = load_model(models_path+"cnn.h5")
pred = model.predict(test_pad_sequences)
y_pred = np.argmax(pred,axis=1)
unique_labels = y_true.unique()
num_classes = len(unique_labels)
# Define target names based on the number of classes
if num_classes == 3:
    target_names = ['class_0', 'class_1', 'class_2']
elif num_classes == 2:
    target_names = ['class_0', 'class_1']
else:
    target_names = [f'class_{i}' for i in range(num_classes)]
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

**Multimodal (Late Fusion)**

In [ ]:
def visual_models():
  '''
      Input: Pretrained Models weight
  '''
  ## ResNet50
  resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  resnet_model = resnet
  resnet_model.trainable = False
  resnet_y = resnet_model.output
  resnet_pool = GlobalAveragePooling2D()(resnet_y)
  resnet_output = Dense(2, activation='softmax')(resnet_pool)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet_model.input, outputs=resnet_output)

  ## VGG16
  vgg16 = VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  vgg16_model = vgg16
  vgg16_model.trainable = False
  vgg16_y = vgg16_model.output
  vgg16_pool = GlobalAveragePooling2D()(vgg16_y)
  vgg16_output = Dense(2, activation='softmax')(vgg16_pool)
  # this is the model we will train
  vgg16_img_model = Model(inputs=vgg16_model.input, outputs=vgg16_output)

  ## DenseNet
  densenet = DenseNet121(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  densenet_model = densenet
  densenet_model.trainable = False
  densenet_y = densenet_model.output
  densenet_pool = GlobalAveragePooling2D()(densenet_y)
  densenet_output = Dense(2, activation='softmax')(densenet_pool)
  # this is the model we will train
  densenet_img_model = Model(inputs=densenet_model.input, outputs=densenet_output)

  ## Inception V3
  inception = InceptionV3(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  inception_model = inception
  inception_model.trainable = False
  inception_y = inception_model.output
  inception_pool = GlobalAveragePooling2D()(inception_y)
  inception_output = Dense(2, activation='softmax')(inception_pool)
  # this is the model we will train
  inception_img_model = Model(inputs=inception_model.input, outputs=inception_output)

  image_models = [resnet_img_model,densenet_img_model,vgg16_img_model,inception_model]

  return image_models


def textual_models():

  ###### BiLSTM Model #######
  bi_text_inputs = Input(shape=(max_len,))
  bi_embedding_layer = Embedding(vocab_size,64)(bi_text_inputs)
  LSTM_Layer_1 = Bidirectional(LSTM(32))(bi_embedding_layer)
  bi_dense_layer_1 = Dense(3, activation='softmax')(LSTM_Layer_1)
  bilstm_model = Model(inputs=bi_text_inputs, outputs=bi_dense_layer_1)


  ### CNN
  cnn_text_inputs = Input(shape=(max_len,))
  cnn_embedding_layer = Embedding(vocab_size,64)(cnn_text_inputs)
  cnn_conv1 = Conv1D(64,2,activation='relu')(cnn_embedding_layer)
  cnn_conv2 = Conv1D(32,2,activation='relu')(cnn_conv1)
  cnn_pool1 = MaxPooling1D(2)(cnn_conv2)

  cnn_flat = Flatten()(cnn_pool1)
  cnn_dense_layer_1 = Dense(3, activation='softmax')(cnn_flat)
  cnn_model = Model(inputs=cnn_text_inputs, outputs=cnn_dense_layer_1)



  ##### LSTM-CNN Model ####
  lc_text_inputs = Input(shape=(max_len,))
  lc_embedding_layer = Embedding(vocab_size,64)(lc_text_inputs)
  LSTM_Layer = Bidirectional(LSTM(32,return_sequences=True))(lc_embedding_layer)
  lc_conv1 = Conv1D(32,2,activation='relu')(LSTM_Layer)
  lc_pool1 = MaxPooling1D(2)(lc_conv1)
  lc_flat = Flatten()(lc_pool1)
  lc_dense_layer_1 = Dense(3, activation='softmax')(lc_flat)
  cnn_lstm_model = Model(inputs=lc_text_inputs, outputs=lc_dense_layer_1)

  ############
  dl_textual_models = [bilstm_model,cnn_model,cnn_lstm_model]

  return dl_textual_models

In [ ]:
visual_model_name = ['ResNet','DenseNet','Vgg16','Inception']
visual_models_lsit = visual_models()

dl_textual_model_name = ['LSTM','CNN','LSTM+CNN']
dl_textual_models_lsit = textual_models()

Resnet

In [ ]:
for tm,textual_model in enumerate(dl_textual_models_lsit):
  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([visual_models_lsit[0].output, textual_model.output])
  final_dense = Dense(4, activation="relu")(con_layer)
  #dropout = Dropout(0.1)(final_dense)
  out = Dense(3,activation='softmax')(final_dense)
  com_model = Model(inputs = [visual_models_lsit[0].input, textual_model.input], outputs=out)

  com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


  print("\nModel Name:=====>\n",visual_model_name[0]+'_'+dl_textual_model_name[tm]+'_late')

  com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[0]+'_'+dl_textual_model_name[tm]+'_late')
            )

In [ ]:
for tm,textual_model in enumerate(dl_textual_models_lsit):
  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([visual_models_lsit[0].output, textual_model.output])
  # Provide a unique name for this Dense layer using tm in the name
  final_dense = Dense(4, activation="relu", name=f"final_dense_{tm}")(con_layer)
  #dropout = Dropout(0.1)(final_dense)
  # Provide a unique name for the output layer using tm in the name
  out = Dense(3,activation='softmax', name=f"output_layer_{tm}")(final_dense)
  com_model = Model(inputs = [visual_models_lsit[0].input, textual_model.input], outputs=out)

  com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


  print("\nModel Name:=====>\n",visual_model_name[0]+'_'+dl_textual_model_name[tm]+'_late')

  com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[0]+'_'+dl_textual_model_name[tm]+'_late')
            )

Densenet

In [ ]:
for tm,textual_model in enumerate(dl_textual_models_lsit):
  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([visual_models_lsit[1].output, textual_model.output])
  # Provide a unique name for this Dense layer using tm in the name
  final_dense = Dense(4, activation="relu", name=f"final_dense_{tm}")(con_layer)
  #dropout = Dropout(0.1)(final_dense)
  # Provide a unique name for the output layer using tm in the name
  out = Dense(3,activation='softmax', name=f"output_layer_{tm}")(final_dense)
  com_model = Model(inputs = [visual_models_lsit[1].input, textual_model.input], outputs=out)

  com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


  print("\nModel Name:=====>\n",visual_model_name[1]+'_'+dl_textual_model_name[tm]+'_late')

  com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[1]+'_'+dl_textual_model_name[tm]+'_late')
            )

Vgg

In [ ]:
for tm,textual_model in enumerate(dl_textual_models_lsit):
  # Concatenating the output of 2 classifiers
  con_layer = keras.layers.concatenate([visual_models_lsit[2].output, textual_model.output])
  # Provide a unique name for this Dense layer using tm in the name
  final_dense = Dense(4, activation="relu", name=f"final_dense_{tm}")(con_layer)
  #dropout = Dropout(0.1)(final_dense)
  # Provide a unique name for the output layer using tm in the name
  out = Dense(3,activation='softmax', name=f"output_layer_{tm}")(final_dense)
  com_model = Model(inputs = [visual_models_lsit[2].input, textual_model.input], outputs=out)

  com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


  print("\nModel Name:=====>\n",visual_model_name[2]+'_'+dl_textual_model_name[tm]+'_late')

  com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[2]+'_'+dl_textual_model_name[tm]+'_late')
            )

Inception

In [ ]:
for tm,textual_model in enumerate(dl_textual_models_lsit):
  # Concatenating the output of 2 classifiers
  # Reshape or flatten the output of one or both models to make them compatible for concatenation
  visual_output = keras.layers.Flatten()(visual_models_lsit[3].output) # Flatten the visual model output
  con_layer = keras.layers.concatenate([visual_output, textual_model.output])
  # Provide a unique name for this Dense layer using tm in the name
  final_dense = Dense(4, activation="relu", name=f"final_dense_{tm}")(con_layer)
  #dropout = Dropout(0.1)(final_dense)
  # Provide a unique name for the output layer using tm in the name
  out = Dense(3,activation='softmax', name=f"output_layer_{tm}")(final_dense)
  com_model = Model(inputs = [visual_models_lsit[3].input, textual_model.input], outputs=out)

  com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


  print("\nModel Name:=====>\n",visual_model_name[3]+'_'+dl_textual_model_name[tm]+'_late')

  com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[3]+'_'+dl_textual_model_name[tm]+'_late')
            )

In [ ]:
for vm in visual_model_name:
  for tm in dl_textual_model_name:
    print('Model name ===> ',vm+'_'+tm)
    model = load_model(models_path+ vm+'_'+tm+'_late'+".h5")
    pred = model.predict([test_image,test_pad_sequences])
    y_pred = np.argmax(pred,axis=1)
    print_metrices(test_data['label'], y_pred)

**Multimodal (Early Fusion)**

In [ ]:
def visual_models():
  '''
      Input: Pretrained Models weight
  '''
  ## ResNet50
  resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  resnet_model = resnet
  resnet_model.trainable = False
  resnet_y = resnet_model.output
  resnet_pool = GlobalAveragePooling2D()(resnet_y)
  resnet_output = Dense(10, activation='relu')(resnet_pool)
  # this is the model we will train
  resnet_img_model = Model(inputs=resnet_model.input, outputs=resnet_output)

  ## Vgg16
  vgg16 = VGG16(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  vgg16_model = vgg16
  vgg16_model.trainable = False
  vgg16_y = vgg16_model.output
  vgg16_pool = GlobalAveragePooling2D()(vgg16_y)
  vgg16_output = Dense(10, activation='relu')(vgg16_pool)
  # this is the model we will
  vgg16_img_model = Model(inputs=vgg16_model.input, outputs=vgg16_output)

  ## DenseNet
  densenet = DenseNet121(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  densenet_model = densenet
  densenet_model.trainable = False
  densenet_y = densenet_model.output
  densenet_pool = GlobalAveragePooling2D()(densenet_y)
  densenet_output = Dense(10, activation='relu')(densenet_pool)
  # this is the model we will train
  densenet_img_model = Model(inputs=densenet_model.input, outputs=densenet_output)

  ## Inception v3
  inception = InceptionV3(weights='imagenet', include_top=False,input_shape=(150, 150, 3))
  inception_model = inception
  inception_model.trainable = False
  inception_y = inception_model.output
  inception_pool = GlobalAveragePooling2D()(inception_y)
  inception_output = Dense(10, activation='relu')(inception_pool)
  # this is the model we will train
  inception_img_model = Model(inputs=inception_model.input, outputs=inception_output)

  #image_models = [inception_model, resnet_img_model,densenet_img_model, vgg16_img_model]
  image_models = [resnet_img_model,densenet_img_model, vgg16_img_model,inception_model]

  return image_models


def textual_models():

  ###### BiLSTM Model #######
  bi_text_inputs = Input(shape=(max_len,))
  bi_embedding_layer = Embedding(vocab_size,64)(bi_text_inputs)
  LSTM_Layer_1 = Bidirectional(LSTM(32))(bi_embedding_layer)
  bi_dense_layer_1 = Dense(10, activation='relu')(LSTM_Layer_1)
  bilstm_model = Model(inputs=bi_text_inputs, outputs=bi_dense_layer_1)


  ### CNN
  cnn_text_inputs = Input(shape=(max_len,))
  cnn_embedding_layer = Embedding(vocab_size,64)(cnn_text_inputs)
  cnn_conv1 = Conv1D(64,2,activation='relu')(cnn_embedding_layer)
  cnn_conv2 = Conv1D(32,2,activation='relu')(cnn_conv1)
  cnn_pool1 = MaxPooling1D(2)(cnn_conv2)

  cnn_flat = Flatten()(cnn_pool1)
  cnn_dense_layer_1 = Dense(10, activation='relu')(cnn_flat)
  cnn_model = Model(inputs=cnn_text_inputs, outputs=cnn_dense_layer_1)



  ##### LSTM-CNN Model ####
  lc_text_inputs = Input(shape=(max_len,))
  lc_embedding_layer = Embedding(vocab_size,64)(lc_text_inputs)
  LSTM_Layer = Bidirectional(LSTM(32,return_sequences=True))(lc_embedding_layer)
  lc_conv1 = Conv1D(32,2,activation='relu')(LSTM_Layer)
  lc_pool1 = MaxPooling1D(2)(lc_conv1)
  lc_flat = Flatten()(lc_pool1)
  lc_dense_layer_1 = Dense(10, activation='relu')(lc_flat)
  cnn_lstm_model = Model(inputs=lc_text_inputs, outputs=lc_dense_layer_1)

  ############
  dl_textual_models = [bilstm_model,cnn_model,cnn_lstm_model]

  return dl_textual_models

In [ ]:
#visual_model_name = ['Inception', 'ResNet','DenseNet','Vgg16']
visual_model_name = ['ResNet','DenseNet','Vgg16','Inception']
visual_models_lsit = visual_models()

dl_textual_model_name = ['LSTM','CNN','LSTM+CNN']
dl_textual_models_lsit = textual_models()

In [ ]:
for vm,visual_model in enumerate(visual_models_lsit):
  for tm,textual_model in enumerate(dl_textual_models_lsit):
    # Concatenating the output of 2 classifiers
    # Flatten the visual model output to make it 2D
    visual_flat = Flatten()(visual_model.output)
    con_layer = keras.layers.concatenate([visual_flat, textual_model.output])

    #final_dense = Dense(10, activation="relu")(con_layer)
    #dropout = Dropout(0.01)(final_dense)
    out = Dense(2,activation='softmax')(con_layer)
    com_model = Model(inputs = [visual_model.input, textual_model.input], outputs=out)

    com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics = ["accuracy"])


    print("\nModel Name:=====>\n",visual_model_name[vm]+'_'+dl_textual_model_name[tm]+'_early')

    com_model.fit([train_image,train_pad_sequences],
                      train_data['label'],
                      validation_split = 0.1,
                      epochs=20,
                      batch_size=32,
                      callbacks = callbacks_check(visual_model_name[vm]+'_'+dl_textual_model_name[tm])
            )

In [ ]:
for vm in visual_model_name:
  for tm in dl_textual_model_name:
    print('Model name ===> ',vm+'_'+tm)
    model = load_model(models_path+ vm+'_'+tm+".h5")
    pred = model.predict([test_image,test_pad_sequences])
    y_pred = np.argmax(pred,axis=1)
    print_metrices(test_data['label'], y_pred)

**Proposed**

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense, Flatten, Conv1D,
                                     MaxPooling1D, LSTM, Bidirectional, Embedding, BatchNormalization,
                                     Dropout, concatenate)
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def visual_models():
    inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    inception_model = inception
    inception_model.trainable = False
    inception_y = inception_model.output
    inception_pool = GlobalAveragePooling2D()(inception_y)
    inception_bn = BatchNormalization()(inception_pool)
    inception_dense1 = Dense(256, activation='relu')(inception_bn)
    inception_dropout1 = Dropout(0.5)(inception_dense1)
    inception_dense2 = Dense(128, activation='relu')(inception_dropout1)
    inception_dropout2 = Dropout(0.5)(inception_dense2)
    inception_output = Dense(10, activation='relu')(inception_dropout2)
    inception_img_model = Model(inputs=inception_model.input, outputs=inception_output)
    image_models = [inception_img_model]
    return image_models


def textual_models():
    lc_text_inputs = Input(shape=(max_len,))
    lc_embedding_layer = Embedding(vocab_size, 64)(lc_text_inputs)
    LSTM_Layer = Bidirectional(LSTM(32, return_sequences=True))(lc_embedding_layer)
    lc_conv1 = Conv1D(64, 2, activation='relu')(LSTM_Layer)
    lc_bn1 = BatchNormalization()(lc_conv1)
    lc_pool1 = MaxPooling1D(2)(lc_bn1)
    lc_conv2 = Conv1D(64, 2, activation='relu')(lc_pool1)
    lc_bn2 = BatchNormalization()(lc_conv2)
    lc_pool2 = MaxPooling1D(2)(lc_bn2)
    lc_flat = Flatten()(lc_pool2)
    lc_dense_layer_1 = Dense(128, activation='relu')(lc_flat)
    lc_dropout1 = Dropout(0.5)(lc_dense_layer_1)
    lc_dense_layer_2 = Dense(10, activation='relu')(lc_dropout1)
    cnn_lstm_model = Model(inputs=lc_text_inputs, outputs=lc_dense_layer_2)
    dl_textual_models = [cnn_lstm_model]
    return dl_textual_models

In [ ]:
def visualize_model(model, model_name):
    plot_model(model, to_file=f'{model_name}.png', show_shapes=True, show_layer_names=True)
    img = mpimg.imread(f'{model_name}.png')
    plt.figure(figsize=(100, 100))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
visual_model_name = ['InceptionV3']
visual_models_lsit = visual_models()

dl_textual_model_name = ['LSTM+CNN']
dl_textual_models_lsit = textual_models()

In [ ]:
for vm, visual_model in enumerate(visual_models_lsit):
    for tm, textual_model in enumerate(dl_textual_models_lsit):
        con_layer = concatenate([visual_model.output, textual_model.output])
        fusion_dense1 = Dense(256, activation='relu')(con_layer)
        fusion_bn1 = BatchNormalization()(fusion_dense1)
        fusion_dropout1 = Dropout(0.5)(fusion_bn1)
        fusion_dense2 = Dense(128, activation='relu')(fusion_dropout1)
        fusion_bn2 = BatchNormalization()(fusion_dense2)
        fusion_dropout2 = Dropout(0.5)(fusion_bn2)
        out = Dense(2, activation='softmax')(fusion_dropout2)
        com_model = Model(inputs=[visual_model.input, textual_model.input], outputs=out)

        com_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=["accuracy"])

        model_name = visual_model_name[vm] + '_proposed' + dl_textual_model_name[tm] + '_early'
        print("\nModel Name:=====>\n", model_name)

        # print("\nModel Name:=====>\n", visual_model_name[vm] + '_proposed' + dl_textual_model_name[tm] + '_early')

        visualize_model(com_model, model_name)